In [8]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from codecarbon import EmissionsTracker
from ptflops import get_model_complexity_info
import os
import pandas as pd
import gc

In [9]:
# ----------------------------
# Clean GPU Memory
# ----------------------------
torch.cuda.empty_cache()
gc.collect()

# ----------------------------
# Device Setup
# ----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [10]:
# ----------------------------
# ShuffleNetV2 Model Definition
# ----------------------------
class ShuffleNetV2_Model(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.base_model = models.shufflenet_v2_x1_0(weights="IMAGENET1K_V1")
        in_features = self.base_model.fc.in_features
        self.base_model.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

In [11]:
# ----------------------------
# Dataset (Memory Safe)
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((160, 160)),   # smaller image for memory efficiency
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset   = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [12]:
# ----------------------------
# Model Setup
# ----------------------------
num_classes = 10
model = ShuffleNetV2_Model(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
# ----------------------------
# FLOPs & Params
# ----------------------------
macs, params = get_model_complexity_info(model, (3, 64, 64),
                                         as_strings=True,
                                         print_per_layer_stat=False,
                                         verbose=False)
macs_value = macs.replace('Mac', '').strip()
if 'G' in macs_value:
    macs_num = float(macs_value.replace('G', '')) * 2
    flops_per_forward = f"{macs_num:.2f} GFLOPs"
elif 'M' in macs_value:
    macs_num = float(macs_value.replace('M', '')) / 1000 * 2
    flops_per_forward = f"{macs_num:.3f} GFLOPs"
else:
    flops_per_forward = f"{macs_value} MACs"

print(f"FLOPs per forward pass: {flops_per_forward} | Params: {params}")

FLOPs per forward pass: 0.025 GFLOPs | Params: 1.26 M


In [14]:
# ----------------------------
# Training & Validation
# ----------------------------
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(loader.dataset)

def validate(model, loader, criterion):
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    acc = 100 * correct / total
    return val_loss / total, acc

In [15]:
# ----------------------------
# Energy & CO₂ Tracking
# ----------------------------
output_dir = "./emissions_shufflenetv2_no_tuning"
os.makedirs(output_dir, exist_ok=True)

tracker = EmissionsTracker(
    project_name="ShuffleNetV2_CIFAR10_NoTuning",
    output_dir=output_dir,
    measure_power_secs=1
)
tracker.start()
start_time = time.time()

[codecarbon WARNING @ 02:29:54] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 02:29:54] [setup] RAM Tracking...
[codecarbon INFO @ 02:29:54] [setup] CPU Tracking...
[codecarbon WARNING @ 02:29:55] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 02:29:55] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 02:29:55] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 02:29:55] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 02:29:55] [setup] GPU Tracking...
[codecarbon INFO @ 02:29:55] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 02:29:55] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation mode

In [16]:
# ----------------------------
# Training Loop (4 Epochs)
# ----------------------------
for epoch in range(4):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)
    print(f"Epoch {epoch+1}/4 | Train Loss: {train_loss:.4f} | Val Acc: {val_acc:.2f}%")


[codecarbon INFO @ 02:30:46] Energy consumed for RAM : 0.000140 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:30:46] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:30:46] Energy consumed for All CPU : 0.000598 kWh
[codecarbon INFO @ 02:30:46] Energy consumed for all GPUs : 0.000449 kWh. Total GPU Power : 32.07353520821586 W
[codecarbon INFO @ 02:30:46] 0.001188 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:30:47] Energy consumed for RAM : 0.000143 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:30:47] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:30:47] Energy consumed for All CPU : 0.000610 kWh
[codecarbon INFO @ 02:30:47] Energy consumed for all GPUs : 0.000459 kWh. Total GPU Power : 37.421911709232866 W
[codecarbon INFO @ 02:30:47] 0.001212 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:3

Epoch 1/4 | Train Loss: 0.5280 | Val Acc: 89.49%


[codecarbon INFO @ 02:35:20] Energy consumed for RAM : 0.000891 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:35:20] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:35:20] Energy consumed for All CPU : 0.003792 kWh
[codecarbon INFO @ 02:35:20] Energy consumed for all GPUs : 0.004670 kWh. Total GPU Power : 46.95818133206265 W
[codecarbon INFO @ 02:35:20] 0.009353 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:35:21] Energy consumed for RAM : 0.000893 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:35:21] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:35:21] Energy consumed for All CPU : 0.003804 kWh
[codecarbon INFO @ 02:35:21] Energy consumed for all GPUs : 0.004687 kWh. Total GPU Power : 60.79378330240322 W
[codecarbon INFO @ 02:35:21] 0.009384 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:35

Epoch 2/4 | Train Loss: 0.2544 | Val Acc: 91.31%


[codecarbon INFO @ 02:40:04] Energy consumed for RAM : 0.001667 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:40:04] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:40:04] Energy consumed for All CPU : 0.007100 kWh
[codecarbon INFO @ 02:40:04] Energy consumed for all GPUs : 0.009008 kWh. Total GPU Power : 47.76965434780592 W
[codecarbon INFO @ 02:40:04] 0.017775 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:40:05] Energy consumed for RAM : 0.001670 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:40:05] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:40:05] Energy consumed for All CPU : 0.007112 kWh
[codecarbon INFO @ 02:40:05] Energy consumed for all GPUs : 0.009024 kWh. Total GPU Power : 58.60346953264059 W
[codecarbon INFO @ 02:40:05] 0.017806 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:40

Epoch 3/4 | Train Loss: 0.1868 | Val Acc: 91.05%


[codecarbon INFO @ 02:44:50] Energy consumed for RAM : 0.002450 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:44:50] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:44:50] Energy consumed for All CPU : 0.010433 kWh
[codecarbon INFO @ 02:44:50] Energy consumed for all GPUs : 0.013382 kWh. Total GPU Power : 48.06436824534203 W
[codecarbon INFO @ 02:44:50] 0.026265 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:44:51] Energy consumed for RAM : 0.002453 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:44:51] Delta energy consumed for CPU with constant : 0.000012 kWh, power : 42.5 W
[codecarbon INFO @ 02:44:51] Energy consumed for All CPU : 0.010444 kWh
[codecarbon INFO @ 02:44:51] Energy consumed for all GPUs : 0.013398 kWh. Total GPU Power : 57.21595281959137 W
[codecarbon INFO @ 02:44:51] 0.026295 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:44

Epoch 4/4 | Train Loss: 0.1549 | Val Acc: 91.54%


In [17]:
# ----------------------------
# Stop Tracker & Show Results
# ----------------------------
emissions = tracker.stop()
elapsed = time.time() - start_time

csv_file = os.path.join(output_dir, "emissions.csv")
df = pd.read_csv(csv_file)
energy = df['energy_consumed'].iloc[-1]  # kWh

print("\n==============================")
print("✅ Final Results Summary (ShuffleNetV2)")
print("==============================")
print(f"CO₂ Emissions : {emissions:.6f} kg")
print(f"Energy Used   : {energy:.6f} kWh")
print(f"Time Taken    : {elapsed/60:.2f} min")
print(f"FLOPs         : {flops_per_forward}")
print("==============================")


[codecarbon INFO @ 02:50:05] Energy consumed for RAM : 0.003312 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 02:50:05] Delta energy consumed for CPU with constant : 0.000011 kWh, power : 42.5 W
[codecarbon INFO @ 02:50:05] Energy consumed for All CPU : 0.014105 kWh
[codecarbon INFO @ 02:50:06] Energy consumed for all GPUs : 0.018008 kWh. Total GPU Power : 33.09345466184562 W
[codecarbon INFO @ 02:50:06] 0.035424 kWh of electricity and 0.000000 L of water were used since the beginning.



✅ Final Results Summary (ShuffleNetV2)
CO₂ Emissions : 0.012371 kg
Energy Used   : 0.035424 kWh
Time Taken    : 20.18 min
FLOPs         : 0.025 GFLOPs
